In [ ]:
!pip install gensim
from gensim.models import KeyedVectors

# Carregando base de textos (IMDB)

In [ ]:
# https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/graduação/mineracao_dados_nao_estruturados/IMDB Dataset.csv',encoding='iso8859')
dataset

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# modelo pré-treinado

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-09-23 23:47:45--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.114.197
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.114.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  64.0MB/s    in 27s     

2021-09-23 23:48:12 (58.7 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



## Carregando o modelo pré-treinado

In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz' # from above
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
palavra = 'investment'
palavra_default_1= word2vec.wv.most_similar(palavra, topn=10) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
palavra = 'oscar'
palavra_default_2= word2vec.wv.most_similar(palavra, topn=10) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


## Cosseno para cálculo de similaridade

In [ ]:
import numpy as np

def cos(x1, x2):
  return np.dot(x1, x2)/(np.linalg.norm(x1)*np.linalg.norm(x2))

## Separando treino e teste

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split


df_train, df_test = train_test_split(dataset, test_size=0.30, random_state=42)

## Classificação usando Média de Word Vectors

In [ ]:
def preprocess(sentence):
    return [w for w in sentence.lower().split() if w not in stop_words]

from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
doc_embeddings = []
for index,row in df_train.iterrows():
  sentence = preprocess(row['review'])
  L = []
  for token in sentence:
    try:
      L.append(word2vec[token])
    except:
      1
  if len(L) > 0: tweet_vec = np.mean(np.array(L),axis=0)
  else: tweet_vec = np.zeros(300)
  doc_embeddings.append(tweet_vec)

In [ ]:
X_train = np.array(doc_embeddings)
y_train = df_train.sentiment.to_list()
X_train.shape

(35000, 300)

In [ ]:
doc_embeddings = []
for index,row in df_test.iterrows():
  sentence = preprocess(row['review'])
  L = []
  for token in sentence:
    try:
      L.append(word2vec[token])
    except:
      1
  if len(L) > 0: tweet_vec = np.mean(np.array(L),axis=0)
  else: tweet_vec = np.zeros(300)
  doc_embeddings.append(tweet_vec)

In [ ]:
X_test = np.array(doc_embeddings)
y_test = df_test.sentiment.to_list()
X_test.shape

(15000, 300)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
score_default = neigh.score(X_test,y_test)

# Treinando sua Word Embeddings

In [ ]:
import gensim 

# percorre cada linha do dataset, preprocessa e armazena na lista documents

documents = []
for index,row in dataset.iterrows(): # para cada linha do dataset
  if len(row['review']) >= 30: # se a linha possui mais do que 30 caracteres
    tokens = gensim.utils.simple_preprocess (row['review']) # preprocessamento simples de cada texto
    documents.append(tokens)

In [ ]:
import numpy as np
len(documents)

50000

In [ ]:
# treinando o Word2Vec a partir dos documents
model = gensim.models.Word2Vec(
        documents, # lista com documents
        size=300, # tamanho da dimensao de cada palavra
        window=2, # tamanho da janela de contexto
        min_count=1, # numero minimo de ocorrencia de uma palavra no texto
        workers=4, # paralelizacao/cpu
        iter=10) # numero maximo de iteracoes

In [ ]:
# testando o modelo..

palavra = 'investment'
palavra_mymodel_1 = model.wv.most_similar(palavra, topn=10) # identificando as 10 palavra mais similares (e.g. cosseno)
palavra_mymodel_1

[('purgation', 0.5754667520523071),
 ('ambitions', 0.5419684052467346),
 ('advancement', 0.5333448648452759),
 ('stimulation', 0.5319089293479919),
 ('exploration', 0.5266944169998169),
 ('aspirations', 0.526160478591919),
 ('convictions', 0.5256863236427307),
 ('turmoils', 0.5256079435348511),
 ('endeavors', 0.5239817500114441),
 ('toxicity', 0.5204833149909973)]

In [ ]:
# testando o modelo..

palavra = 'oscar'
palavra_mymodel_2 = model.wv.most_similar(palavra, topn=10) # identificando as 10 palavra mais similares (e.g. cosseno)

In [ ]:
word2vec = model

## Separando treino e teste
Use a mesma semente do exemplo prático anterior

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split


df_train, df_test = train_test_split(dataset, test_size=0.30, random_state=42)

## Classificação usando Média de Word Vectors

In [ ]:
doc_embeddings = []
for index,row in df_train.iterrows():
  sentence = preprocess(row['review'])
  L = []
  for token in sentence:
    try:
      L.append(word2vec.wv[token])
    except:
      1
  if len(L) > 0: tweet_vec = np.mean(np.array(L),axis=0)
  else: tweet_vec = np.zeros(300)
  doc_embeddings.append(tweet_vec)

In [ ]:
X_train = np.array(doc_embeddings)
y_train = df_train.sentiment.to_list()
X_train.shape

(35000, 300)

In [ ]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.177112,0.079944,0.150981,0.337442,-0.020517,0.127606,0.060465,0.185810,-0.058184,0.041430,0.276693,0.010812,-0.186891,0.330967,0.017373,-0.193129,0.039530,-0.000779,-0.032176,-0.173601,-0.046074,0.009463,0.195432,0.190249,-0.157615,0.047294,-0.086391,-0.078817,-0.016542,-0.072022,0.165116,0.093285,-0.368298,0.383310,-0.189221,0.132296,0.011986,-0.038727,0.108934,-0.035761,...,-0.074624,-0.035830,-0.117313,0.003436,0.074364,-0.206507,0.079455,0.324227,0.193347,0.030348,0.065786,-0.117723,0.178577,0.097984,-0.076375,-0.118968,0.323779,-0.037942,0.209013,-0.334435,-0.074188,0.031632,-0.138700,-0.092997,0.244811,0.217668,-0.044694,0.498606,0.210059,-0.091763,0.032725,-0.060331,-0.044159,-0.183998,0.026079,0.044778,-0.073079,-0.118590,0.135755,-0.022072
1,-0.065182,0.212472,0.048674,-0.141965,-0.071602,0.082386,-0.050302,0.286429,-0.074530,0.032989,0.226338,0.049428,-0.111512,-0.014068,0.098360,-0.251064,0.084894,-0.056995,0.092530,-0.043288,-0.184358,0.071158,-0.125183,0.061311,0.179561,-0.132459,0.060767,0.122266,-0.100010,-0.263858,0.131398,-0.031090,0.257983,0.253938,-0.163050,-0.039945,0.008551,-0.010319,-0.101502,-0.076208,...,0.008579,-0.065682,-0.063070,0.082178,0.119131,0.004655,0.163611,0.106950,0.017165,0.152192,-0.028997,-0.292586,0.102378,0.214958,-0.047939,-0.110458,0.106137,-0.024290,0.135901,-0.220941,-0.048687,-0.065669,-0.045449,0.053226,0.041994,-0.114068,-0.170017,0.350660,0.029547,-0.006624,-0.111192,-0.034870,-0.001097,-0.037132,0.079984,-0.078904,-0.215633,-0.190626,-0.218226,-0.125287
2,-0.172593,0.002091,0.143070,0.258753,0.055257,-0.008075,-0.117416,0.185498,-0.126956,-0.011525,0.327917,0.170238,-0.193891,0.205554,0.038686,-0.298527,0.056281,-0.118258,-0.092517,-0.180273,-0.064069,0.190361,0.098846,0.319055,0.050003,0.098699,0.013596,-0.050147,0.007649,-0.383994,0.105589,-0.113561,-0.119480,0.250080,-0.079233,0.101739,-0.078511,-0.033466,0.156629,-0.110564,...,-0.050311,-0.028409,-0.164440,-0.002958,0.105523,-0.201984,-0.091420,0.387398,0.251011,-0.079348,0.074767,-0.002824,0.168511,0.168959,-0.187304,-0.060522,0.187065,-0.047405,0.177961,-0.290232,-0.131224,0.095102,-0.153494,0.149239,0.262908,0.162429,-0.044794,0.363169,0.062667,-0.113572,0.026579,0.022735,0.094370,-0.242277,0.116097,0.012329,-0.060433,-0.107415,-0.015250,-0.103897
3,-0.018479,0.088322,0.353076,0.545897,0.108551,-0.113232,-0.024457,0.361195,-0.244875,-0.195275,0.612319,0.252634,-0.236251,0.273946,0.249534,-0.404563,0.062159,-0.070136,0.030866,-0.463376,0.294882,0.172566,0.228752,0.606904,-0.334190,-0.013367,-0.153222,-0.232640,-0.048098,-0.770042,0.275087,0.084482,-0.136580,0.448034,-0.082087,0.098878,-0.135764,-0.175160,0.243257,-0.205906,...,-0.249719,-0.088966,-0.232667,0.111140,0.053979,-0.561780,0.273937,0.730080,0.447080,-0.350146,0.294210,0.205585,0.512814,0.693272,-0.421807,-0.516980,0.268901,-0.142701,0.218376,-0.483492,0.155741,0.171832,-0.424412,0.117814,0.990478,0.743781,0.063728,0.968918,0.049521,-0.062768,0.435820,-0.291288,0.011989,-0.336968,0.274350,-0.221307,-0.222559,-0.116888,0.019244,0.259263
4,-0.129192,0.017452,0.242520,0.247117,-0.099983,-0.113728,0.070738,0.280092,0.052384,-0.110948,0.423382,0.316948,-0.174510,0.135090,0.065445,-0.368350,-0.083080,-0.114927,-0.002729,-0.188704,0.169218,0.035249,0.541693,0.197470,-0.128522,0.037240,-0.043271,-0.099754,-0.058259,-0.428800,0.226654,0.088017,-0.277015,0.315324,-0.046459,-0.038452,-0.047458,-0.000415,0.012341,-0.012366,...,-0.161952,0.099357,-0.128674,0.239858,-0.052888,-0.249717,0.231914,0.356709,0.173137,-0.071797,0.245389,-0.022513,0.491729,0.398891,-0.291379,-0.289739,0.224528,0.077017,0.014016,-0.194784,-0.058976,0.082111,-0.139310,-0.033123,0.598875,0.505888,-0.049518,0.556120,-0.109074,-

In [ ]:
doc_embeddings = []
for index,row in df_test.iterrows():
  sentence = preprocess(row['review'])
  L = []
  for token in sentence:
    try:
      #print(token,word2vec[token])
      L.append(word2vec[token])
    except:
      1
  if len(L) > 0: tweet_vec = np.mean(np.array(L),axis=0)
  else: tweet_vec = np.zeros(300)
  doc_embeddings.append(tweet_vec)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
X_test = np.array(doc_embeddings)
y_test = df_test.sentiment.to_list()
X_test.shape

(15000, 300)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
score_mymodel = neigh.score(X_test,y_test)

## Versão com embeddings normalizadas

In [ ]:
length = np.sqrt((X_train**2).sum(axis=1))[:,None]
X_train_norm = X_train / (length+0.00001)

length = np.sqrt((X_test**2).sum(axis=1))[:,None]
X_test_norm = X_test / (length+0.00001)


In [ ]:
pd.DataFrame(X_test)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.041393,0.184601,-0.023300,-0.009399,-0.034013,0.023475,-0.197662,0.115482,-0.003678,0.061023,0.346324,0.086338,-0.034731,0.139671,0.075884,-0.231790,-0.053158,0.003926,0.148864,-0.099789,-0.172956,0.102884,0.035065,0.187854,0.129256,-0.022112,-0.054191,0.125477,-0.119564,-0.270301,0.242932,-0.010440,0.072334,0.310442,-0.159424,0.044620,-0.056447,-0.050659,-0.119916,-0.021397,...,-0.033695,-0.081140,-0.021576,0.103727,0.148833,-0.100651,0.121601,0.174843,0.043175,0.016488,0.002173,-0.216554,0.263085,0.133218,-0.105177,-0.048827,0.088380,0.015531,0.074477,-0.198244,0.015068,0.096472,0.084640,-0.106053,0.130932,0.085202,-0.021544,0.462704,0.041341,0.024050,-0.030088,0.025014,0.027335,-0.061589,0.177670,-0.113534,-0.212132,-0.109381,-0.059540,-0.177861
1,0.131375,-0.010290,0.169625,0.328785,0.012572,-0.174021,-0.013092,0.162796,0.044668,-0.189477,0.400115,0.262370,-0.225839,0.228033,-0.022289,-0.282431,0.206623,-0.107240,0.033323,-0.151735,0.007559,-0.071547,0.387467,0.308888,-0.132540,0.048703,-0.051577,-0.192934,-0.129878,-0.343776,0.264193,-0.067882,-0.186301,0.135437,-0.072676,-0.056181,-0.165819,0.029160,0.107910,-0.086737,...,0.042088,0.004405,-0.200902,0.025595,0.079021,-0.144451,-0.037603,0.480884,0.283631,-0.231234,0.240314,0.117755,0.461914,0.301530,-0.251664,-0.232930,0.354094,0.059967,0.074393,-0.162996,0.020134,0.099185,-0.090424,-0.035641,0.515561,0.364956,-0.052324,0.406844,-0.060599,-0.116106,0.202235,-0.272189,0.135322,-0.173094,0.203580,-0.082058,-0.087474,-0.284328,-0.028953,0.047471
2,-0.058835,0.150008,0.190296,0.243400,0.026427,-0.049675,-0.110144,0.303160,0.009891,-0.075976,0.365463,0.092584,-0.071480,0.346720,0.095333,-0.163302,0.053517,-0.052109,0.099831,-0.249179,0.048678,-0.047099,0.400706,0.271785,-0.149573,0.128508,-0.235398,-0.134168,-0.058650,-0.491183,0.400289,0.038730,-0.288943,0.476431,0.018777,0.067871,-0.048544,-0.016315,-0.077820,0.032381,...,-0.172123,-0.029029,-0.018978,0.320790,-0.098532,-0.241445,0.187417,0.413049,0.291124,-0.022409,0.257129,0.005030,0.357437,0.415336,-0.279757,-0.167161,0.382836,0.038584,0.067431,-0.338486,0.143808,0.084302,-0.107151,-0.085694,0.671605,0.360404,-0.067387,0.752668,0.018447,-0.046551,0.185243,-0.245837,-0.182747,-0.128427,0.021026,-0.104351,-0.041524,-0.135028,0.052446,0.049007
3,-0.058106,-0.065275,0.170943,0.082510,-0.111386,0.179702,0.213744,0.341059,0.031717,-0.102598,0.225577,-0.058246,0.026237,0.181676,-0.003154,-0.240958,-0.047755,-0.023705,-0.105764,-0.169320,0.017189,0.105779,0.009539,0.033129,-0.149666,-0.132954,-0.030262,-0.179743,-0.028069,-0.248568,0.070703,0.008727,0.011837,0.281516,-0.112269,0.141785,-0.082030,-0.033713,0.162481,0.002726,...,-0.387881,-0.311346,-0.246035,-0.058759,0.018212,-0.371312,0.002196,0.244620,0.383316,-0.136042,-0.088428,-0.102574,0.184745,0.255450,-0.203350,-0.072979,0.198063,-0.114722,0.174320,0.045066,0.016283,0.044301,0.081395,0.258003,0.376645,0.247909,-0.016623,0.318434,0.020191,-0.043642,0.006076,-0.131846,0.154010,-0.287170,-0.148583,0.003991,0.001047,0.053827,-0.145556,0.002046
4,-0.042704,0.080146,0.238014,0.232490,-0.027610,0.116655,0.025138,0.298192,-0.166679,-0.093239,0.325049,0.050297,-0.282513,0.409804,0.001723,-0.333929,-0.043559,-0.119013,-0.062089,-0.286484,0.117390,-0.010231,0.457022,0.308259,-0.098423,0.078250,-0.109493,-0.074918,-0.099642,-0.415392,0.130898,0.005212,-0.237938,0.378010,0.051526,0.141640,-0.120511,0.001779,0.012297,-0.111970,...,0.095686,0.066533,-0.193561,0.077747,0.016380,-0.265192,0.179013,0.404495,0.155117,-0.172318,0.154258,0.028826,0.425486,0.241004,-0.247735,-0.228652,0.391616,-0.098572,0.158295,-0.300176,0.103900,0.143251,-0.159708,-0.140775,0.501397,0.441446,0.091465,0.632482,-0.010776,-0.179388,0.20

In [ ]:
pd.DataFrame(X_test_norm)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.016559,0.073848,-0.009321,-0.003760,-0.013606,0.009391,-0.079072,0.046197,-0.001471,0.024411,0.138543,0.034539,-0.013894,0.055874,0.030357,-0.092725,-0.021265,0.001571,0.059551,-0.039919,-0.069189,0.041158,0.014027,0.075149,0.051707,-0.008846,-0.021678,0.050196,-0.047830,-0.108131,0.097182,-0.004177,0.028936,0.124188,-0.063776,0.017850,-0.022581,-0.020266,-0.047971,-0.008560,...,-0.013479,-0.032459,-0.008631,0.041495,0.059539,-0.040264,0.048645,0.069944,0.017272,0.006596,0.000869,-0.086630,0.105244,0.053292,-0.042075,-0.019533,0.035355,0.006213,0.029794,-0.079305,0.006028,0.038593,0.033859,-0.042425,0.052378,0.034084,-0.008618,0.185099,0.016538,0.009621,-0.012037,0.010006,0.010935,-0.024638,0.071075,-0.045418,-0.084861,-0.043757,-0.023818,-0.071151
1,0.037464,-0.002934,0.048372,0.093760,0.003585,-0.049626,-0.003734,0.046425,0.012738,-0.054034,0.114102,0.074821,-0.064403,0.065029,-0.006356,-0.080542,0.058923,-0.030582,0.009503,-0.043271,0.002156,-0.020403,0.110495,0.088086,-0.037797,0.013889,-0.014708,-0.055020,-0.037038,-0.098035,0.075340,-0.019358,-0.053128,0.038623,-0.020725,-0.016021,-0.047287,0.008316,0.030773,-0.024735,...,0.012002,0.001256,-0.057292,0.007299,0.022534,-0.041193,-0.010723,0.137135,0.080884,-0.065942,0.068531,0.033581,0.131725,0.085988,-0.071768,-0.066425,0.100978,0.017101,0.021215,-0.046482,0.005742,0.028285,-0.025786,-0.010164,0.147024,0.104075,-0.014921,0.116021,-0.017281,-0.033110,0.057672,-0.077621,0.038590,-0.049362,0.058055,-0.023401,-0.024945,-0.081082,-0.008256,0.013537
2,-0.014411,0.036742,0.046610,0.059618,0.006473,-0.012167,-0.026978,0.074255,0.002423,-0.018609,0.089515,0.022677,-0.017508,0.084924,0.023350,-0.039999,0.013108,-0.012763,0.024452,-0.061033,0.011923,-0.011536,0.098147,0.066570,-0.036636,0.031476,-0.057658,-0.032863,-0.014366,-0.120309,0.098045,0.009486,-0.070773,0.116695,0.004599,0.016624,-0.011890,-0.003996,-0.019061,0.007931,...,-0.042159,-0.007110,-0.004648,0.078573,-0.024134,-0.059139,0.045905,0.101171,0.071307,-0.005489,0.062980,0.001232,0.087549,0.101731,-0.068523,-0.040944,0.093771,0.009451,0.016516,-0.082908,0.035224,0.020649,-0.026245,-0.020990,0.164500,0.088276,-0.016505,0.184356,0.004518,-0.011402,0.045373,-0.060214,-0.044761,-0.031457,0.005150,-0.025559,-0.010171,-0.033073,0.012846,0.012004
3,-0.018846,-0.021172,0.055445,0.026762,-0.036128,0.058286,0.069327,0.110622,0.010287,-0.033277,0.073166,-0.018892,0.008510,0.058926,-0.001023,-0.078154,-0.015489,-0.007689,-0.034304,-0.054919,0.005575,0.034309,0.003094,0.010745,-0.048544,-0.043124,-0.009816,-0.058299,-0.009104,-0.080623,0.022932,0.002831,0.003839,0.091309,-0.036414,0.045988,-0.026606,-0.010935,0.052701,0.000884,...,-0.125809,-0.100985,-0.079801,-0.019058,0.005907,-0.120434,0.000712,0.079342,0.124328,-0.044125,-0.028681,-0.033270,0.059922,0.082855,-0.065956,-0.023671,0.064242,-0.037210,0.056541,0.014617,0.005281,0.014369,0.026400,0.083683,0.122164,0.080409,-0.005392,0.103284,0.006549,-0.014155,0.001971,-0.042764,0.049953,-0.093143,-0.048193,0.001295,0.000340,0.017459,-0.047211,0.000664
4,-0.012291,0.023069,0.068508,0.066918,-0.007947,0.033577,0.007236,0.085829,-0.047976,-0.026837,0.093559,0.014477,-0.081316,0.117955,0.000496,-0.096115,-0.012538,-0.034256,-0.017871,-0.082459,0.033789,-0.002945,0.131545,0.088727,-0.028329,0.022523,-0.031515,-0.021564,-0.028680,-0.119563,0.037677,0.001500,-0.068486,0.108803,0.014831,0.040769,-0.034687,0.000512,0.003539,-0.032229,...,0.027542,0.019150,-0.055713,0.022378,0.004715,-0.076331,0.051526,0.116426,0.044648,-0.049599,0.044400,0.008297,0.122468,0.069369,-0.071306,-0.065813,0.112719,-0.028372,0.045562,-0.086400,0.029906,0.041232,-0.045969,-0.040519,0.144318,0.127062,0.026326,0.182048,-0.003102,-0.051633,0.05

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh_norm = KNeighborsClassifier(n_neighbors=3)
neigh_norm.fit(X_train_norm, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
score_mymodel_norm = neigh_norm.score(X_test_norm,y_test)

# Conclusão

Para uma palavra fora do escopo do nosso dataset vemos que o modelo genérico nos fornece palavras muito mais semelhantes que o modelo próprio, o inverso acontece quando usamos uma palavra que está diferetamente relacionado ao dataset, como é o caso de "Oscar"



In [ ]:
d = {
    "Modelo genérico (Investment)": palavra_default_1,
    "Modelo próprio (Investment)": palavra_mymodel_1,
    "Modelo genérico (Oscar)": palavra_default_2,
    "Modelo próprio (Oscar)": palavra_mymodel_2
}
df = pd.DataFrame(data = d)
df

,Modelo genérico (Investment),Modelo próprio (Investment),Modelo genérico (Oscar),Modelo próprio (Oscar)
0,"(investments, 0.8098693490028381)","(purgation, 0.5754667520523071)","(oscars, 0.6721882224082947)","(award, 0.7586902976036072)"
1,"(investing, 0.7012243270874023)","(ambitions, 0.5419684052467346)","(emmy, 0.5937102437019348)","(emmy, 0.7499661445617676)"
2,"(Investment, 0.6812148094177246)","(advancement, 0.5333448648452759)","(Oscar, 0.5813542008399963)","(nomination, 0.6745911836624146)"
3,"(invesment, 0.6630898714065552)","(stimulation, 0.5319089293479919)","(grammy, 0.5810931921005249)","(bafta, 0.6492618918418884)"
4,"(investor, 0.6319271326065063)","(exploration, 0.5266944169998169)","(Oscar_nom, 0.5700913667678833)","(oscars, 0.6448333263397217)"
5,"(invest, 0.6203395128250122)","(aspirations, 0.526160478591919)","(kristen_stewart, 0.5589216351509094)","(razzie, 0.6437212824821472)"
6,"(investors, 0.5946779847145081)","(convictions, 0.5256863236427307)","(Academy_Award, 0.5582292675971985)","(awards, 0.614215612411499)"
7,"(equity, 0.5897506475448608)","(turmoils, 0.5256079435348511)","(robert_pattinson, 0.5571187734603882)","(academy, 0.6055071353912354)"
8,"(investement, 0.5615226030349731)","(endeavors, 0.5239817500114441)","(nom, 0.5552184581756592)","(nominations, 0.580274224281311)"
9,"(Investments, 0.5592135190963745)","(toxicity, 0.5204833149909973)","(Oscars, 0.553540825843811)","(prize, 0.5631612539291382)"


Verifiquei se o modelo que eu treinei prevê melhor os dados da base do que o modelo pré-treinado, dado os resultados abaixo não encontramos diferença significava. Porém não se deve deixar de lado a conclusão anterior, na qual o modelo próprio foi muito mais eficaz para encontrar palavras similares dentro do assunto do dataset.

In [ ]:
d_2 = {
    "Modelo genérico": [score_default],
    "Modelo próprio": [score_mymodel],
    "Modelo próprio normalizado": [score_mymodel_norm]
}
df_2 = pd.DataFrame(data = d_2)
df_2

,Modelo genérico,Modelo próprio,Modelo próprio normalizado
0,0.753867,0.744267,0.7548
